# Import package

In [20]:
#Install module
# %pip install --user --upgrade pandas
%pip install --user SQLAlchemy

  Obtaining dependency information for SQLAlchemy from https://files.pythonhosted.org/packages/2d/eb/46ea0ba0db3a84b5971cc258f13ee4ac2b12ed8198823f8ba7a03ab00ead/SQLAlchemy-2.0.19-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for typing-extensions>=4.2.0 from https://files.pythonhosted.org/packages/ec/6b/63cc3df74987c36fe26157ee12e09e8f9db4de771e0f3404263117e75b95/typing_extensions-4.7.1-py3-none-any.whl.metadata
     ---------------------------------------- 0.0/192.5 kB ? eta -:--:--
     ------------------------------------- 192.5/192.5 kB 11.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   ----------- ---------------------------- 0.6/2.0 MB 17.8 MB/s eta 0:00:01
   ---------------------- ----------------- 1.1/2.0 MB 14.0 MB/s eta 0:00:01
   ---------------------------------- ----- 1.7/2.0 MB 15.7 MB/s eta 0:00:01
   ---------------------------------------- 2.0/2.0 MB 14.0 MB/s eta 0:00:00
Note: you may need to restart

In [21]:
#Import package
import numpy as np
import pandas as pd
from pandas_datareader import data as web
import requests

import sqlite3 #Connect SQLite
from sqlalchemy.type import Integer, Numeric, Date
# import os
# from tqdm import tqdm
# from datetime import datetime
# import yfinance as yf
# import lxml
# from bs4 import BeautifulSoup #BeutifulSoup

ModuleNotFoundError: No module named 'sqlalchemy.type'

# Connect SQLite3 Database

In [9]:
#Connect SQLite
import sqlite3

conn = sqlite3.connect('StockData.db')

# Some useful functions

In [10]:
#Get Alpha Vantage api key
def get_apikey(filename: str):
    with open(filename) as f:
        api_key = f.read().strip()
    f.close
    return api_key

In [11]:
#Get daily core stock data from Alpha Vantage from 2000-01 to now
def get_daily_stockdata(symbol:str, outputsize = 'compact', datatype = 'json'):
    function = 'TIME_SERIES_DAILY'
    datatype = datatype
    outputsize = outputsize
    ticker = symbol
    alpha_vantage_apikey = get_apikey(filename= 'dist/apikey_AlphaVantage')

    url = f'https://www.alphavantage.co/query?function={function}&symbol={ticker}&outputsize={outputsize}&apikey={alpha_vantage_apikey}&datatype={datatype}'
    r = requests.get(url)
    df = pd.DataFrame.from_dict(r.json()['Time Series (Daily)'],orient='index')
    df.columns = ['Open', 'High', 'Low', 'Close', 'Volume']
    df = df.rename_axis('Date').reset_index()
    df.insert(loc = 0, column = 'Ticker', value = ticker, allow_duplicates=True)
    return df

# Data Preparation

## Web Scrapping

This section will perform web scrapping to scrap all S&P500 tickers on wikipedia. https://en.wikipedia.org/wiki/List_of_S%26P_500_companies

We can simply use padnas to get the table from wiki.

In [12]:
#Store the S&P information in pandas dataframe
wiki_url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies#Selected_changes_to_the_list_of_S&P_500_components"
tickers_df_list = pd.read_html(wiki_url)
tickers_df = tickers_df_list[0]
tickers_df.head()

,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded
0,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,66740,1902
1,AOS,A. O. Smith,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916
2,ABT,Abbott,Health Care,Health Care Equipment,"North Chicago, Illinois",1957-03-04,1800,1888
3,ABBV,AbbVie,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
4,ACN,Accenture,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989


In [13]:
#Turn pandas dataframe to the list
tickers = tickers_df['Symbol'].values.tolist()
print(tickers)

['MMM', 'AOS', 'ABT', 'ABBV', 'ACN', 'ATVI', 'ADM', 'ADBE', 'ADP', 'AAP', 'AES', 'AFL', 'A', 'APD', 'AKAM', 'ALK', 'ALB', 'ARE', 'ALGN', 'ALLE', 'LNT', 'ALL', 'GOOGL', 'GOOG', 'MO', 'AMZN', 'AMCR', 'AMD', 'AEE', 'AAL', 'AEP', 'AXP', 'AIG', 'AMT', 'AWK', 'AMP', 'ABC', 'AME', 'AMGN', 'APH', 'ADI', 'ANSS', 'AON', 'APA', 'AAPL', 'AMAT', 'APTV', 'ACGL', 'ANET', 'AJG', 'AIZ', 'T', 'ATO', 'ADSK', 'AZO', 'AVB', 'AVY', 'AXON', 'BKR', 'BALL', 'BAC', 'BBWI', 'BAX', 'BDX', 'WRB', 'BRK.B', 'BBY', 'BIO', 'TECH', 'BIIB', 'BLK', 'BK', 'BA', 'BKNG', 'BWA', 'BXP', 'BSX', 'BMY', 'AVGO', 'BR', 'BRO', 'BF.B', 'BG', 'CHRW', 'CDNS', 'CZR', 'CPT', 'CPB', 'COF', 'CAH', 'KMX', 'CCL', 'CARR', 'CTLT', 'CAT', 'CBOE', 'CBRE', 'CDW', 'CE', 'CNC', 'CNP', 'CDAY', 'CF', 'CRL', 'SCHW', 'CHTR', 'CVX', 'CMG', 'CB', 'CHD', 'CI', 'CINF', 'CTAS', 'CSCO', 'C', 'CFG', 'CLX', 'CME', 'CMS', 'KO', 'CTSH', 'CL', 'CMCSA', 'CMA', 'CAG', 'COP', 'ED', 'STZ', 'CEG', 'COO', 'CPRT', 'GLW', 'CTVA', 'CSGP', 'COST', 'CTRA', 'CCI', 'CSX', 'C

## Download the financial data

In [14]:
#Download the financial data from Alpha vantage
# tickers_test = ['AAPL','MMM','ABT', 'ABBV', 'ACN', 'ATVI', 'ADM', 'ADBE', 'ADP', 'AAP', 'AES', 'AFL']

stock_df = pd.DataFrame(columns=['Ticker','Date','Open', 'High', 'Low', 'Close', 'Volume'])
for ticker in tickers:
    try: stock_df = pd.concat([stock_df, get_daily_stockdata(ticker,outputsize='full')], ignore_index = True)
    except: continue

In [15]:
stock_df

,Ticker,Date,Open,High,Low,Close,Volume
0,MMM,2023-07-28,111.8700,112.4100,111.0450,111.8800,2910759
1,MMM,2023-07-27,110.4500,113.1400,110.4500,111.1900,5613488
2,MMM,2023-07-26,109.8400,113.0700,109.8400,112.6400,5594986
3,MMM,2023-07-25,107.5500,110.8500,107.0092,109.8300,8778880
4,MMM,2023-07-24,104.5300,105.6041,103.9800,104.2700,3333380
...,...,...,...,...,...,...,...
162218,UDR,1999-11-05,11.0600,11.1900,11.0000,11.1200,219300
162219,UDR,1999-11-04,11.1200,11.1200,11.0000,11.0000,184500
162220,UDR,1999-11-03,11.1200,11.1200,10.9400,11.0000,245200
162221,UDR,1999-11-02,11.0600,11.1900,11.0600,11.0600,244800


In [19]:
#Store the data into SQLite3 database
table_name = 'StockPrice'

stock_df.to_sql(table_name,conn,if_exists='replace',
                index=True,index_label='Record',
                dtype={'Ticker': Text(),
                       'Date'  : Date(),
                       'Open'  : Numeric(),
                       'High'  : Numeric(),
                       'Low'   : Numeric(),
                       'Close' : Numeric(),
                       'Volume': Integer()})
conn.commit()

NameError: name 'Text' is not defined